In [20]:
import os
from pathlib import Path
import requests
from tqdm import tqdm
from typing import Union, List

import math
import torch
import numpy as np
import audiotools
import dasp_pytorch
import auraloss
# import laion_clap
from audiotools import AudioSignal

from transformers import BertForMaskedLM

from scipy import signal
import matplotlib.pyplot as plt

import helper

In [21]:
NOTEBOOKS_DIR = Path(os.path.abspath(''))
PROJECT_DIR = NOTEBOOKS_DIR.parent
ASSETS_DIR = PROJECT_DIR / "assets"
PRETRAINED_DIR = PROJECT_DIR / "pretrained"
DATA_DIR = PROJECT_DIR / "data"
RUNS_DIR = PROJECT_DIR / "runs"
EXPERIMENTS_DIR = PROJECT_DIR / "experiments"
EXP_AUDEALIZE_DIR = EXPERIMENTS_DIR / "audealize_comp"
print(ASSETS_DIR)

/home/annie/research/text2fx/assets


#### Audealize Gnd truth

In [22]:
# Load JSON data Audealize API word <> EQ settings
file_path = '/home/annie/research/text2fx/notebooks/audealize_data/eqdescriptors.json'
top10_eq = ["warm", "cold", "soft", "loud", "happy", "bright", "soothing", "harsh", "heavy", "cool"]
settings_dict = helper.get_settings_for_words(file_path, top10_eq)
# print(settings_dict['cool'])

freq_bands = [20, 50, 83, 120, 161, 208, 259, 318, 383, 455, 537, 628, 729, 843, 971, 
              1114, 1273, 1452, 1652, 1875, 2126, 2406, 2719, 3070, 3462, 3901, 
              4392, 4941, 5556, 6244, 7014, 7875, 8839, 9917, 11124, 12474, 13984, 
              15675, 17566, 19682]

converted_settings_dict = helper.convert_to_freq_gain_tuples(settings_dict, freq_bands)
print(converted_settings_dict['cold'])

# Example usage:
# Assuming `converted_settings` is already defined
tensor_settings = helper.convert_to_tensors(converted_settings_dict)
print(tensor_settings['cold'])

[(20, -0.580513251356788), (50, -0.6834067987276559), (83, -0.773504222665684), (120, -0.7826239592385814), (161, -0.7345700959971675), (208, -0.6566334764439306), (259, -0.5653233608770033), (318, -0.4836950058455057), (383, -0.36607153040596346), (455, -0.21076646565785281), (537, -0.024685418825977225), (628, 0.13490708566568516), (729, 0.24364537837196318), (843, 0.2919493430840968), (971, 0.306169237514744), (1114, 0.3154850504552722), (1273, 0.34026806742471727), (1452, 0.3780090357874527), (1652, 0.4000209159215504), (1875, 0.42316115382137015), (2126, 0.43181439731447013), (2406, 0.4080766047517407), (2719, 0.3790977075339527), (3070, 0.3540513947136643), (3462, 0.3267412208869599), (3901, 0.2654396716724671), (4392, 0.18942117677235806), (4941, 0.13958471969901018), (5556, 0.12210507502213225), (6244, 0.11130983306709169), (7014, 0.10873017389381907), (7875, 0.1109628696217952), (8839, 0.08195184335636668), (9917, 0.004477092398627467), (11124, -0.06096913507489133), (12474, -

In [23]:
# Loading audealize ground truth files
input_raw_piano = helper.load_and_find_path_with_keyword(ASSETS_DIR, ["piano"], returnSingle=True)
input_raw_drums = helper.load_and_find_path_with_keyword(ASSETS_DIR, ["drums"], returnSingle=True)
input_raw_speech = helper.load_and_find_path_with_keyword(ASSETS_DIR, ["225"], returnSingle=True)

print(input_raw_piano)

/home/annie/research/text2fx/assets/audealize_examples/piano.wav


In [28]:
EXPORT_EXAMPLES_DIR = Path(EXP_AUDEALIZE_DIR / "audealize_version")

In [30]:
# Original Input
audio_type = "speech"

#loading clean file
input_raw= helper.load_and_find_path_with_keyword(ASSETS_DIR, [audio_type], returnSingle=True)
input_sig = AudioSignal(input_raw).to_mono()

# generating output files
for word, freq_gains in tensor_settings.items():
    filtered_sig = helper.dasp_apply_multiple_filters_file(input_raw, freq_gains)
    filter_out = AudioSignal(filtered_sig,input_sig.sample_rate)
    print(f'saving {word}')

    EXPORT_EX_DIR = Path(EXPORT_EXAMPLES_DIR / f"{audio_type}")
    EXPORT_EX_DIR.mkdir(exist_ok=True)

    filter_out.write(Path(EXPORT_EX_DIR, f"{word}.wav"))


clean signal...
saving warm
saving cold
saving soft
saving loud
saving happy
saving bright
saving soothing
saving harsh
saving heavy
saving cool
